In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load the dataset
df = pd.read_csv(r"C:\Users\bhara\Documents\Global_Superstore2.csv",encoding="ISO-8859-1")

In [ ]:
df.head()

   Row ID         Order ID  Order Date  ...    Profit Shipping Cost Order Priority
0   32298   CA-2012-124891  31-07-2012  ...  762.1845        933.57       Critical
1   26341    IN-2013-77878  05-02-2013  ... -288.7650        923.63       Critical
2   25330    IN-2013-71249  17-10-2013  ...  919.9710        915.49         Medium
3   13524  ES-2013-1579342  28-01-2013  ...  -96.5400        910.16         Medium
4   47221     SG-2013-4320  05-11-2013  ...  311.5200        903.04       Critical

[5 rows x 24 columns]


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Row ID          51290 non-null  int64  
 1   Order ID        51290 non-null  object 
 2   Order Date      51290 non-null  object 
 3   Ship Date       51290 non-null  object 
 4   Ship Mode       51290 non-null  object 
 5   Customer ID     51290 non-null  object 
 6   Customer Name   51290 non-null  object 
 7   Segment         51290 non-null  object 
 8   City            51290 non-null  object 
 9   State           51290 non-null  object 
 10  Country         51290 non-null  object 
 11  Postal Code     9994 non-null   float64
 12  Market          51290 non-null  object 
 13  Region          51290 non-null  object 
 14  Product ID      51290 non-null  object 
 15  Category        51290 non-null  object 
 16  Sub-Category    51290 non-null  object 
 17  Product Name    51290 non-null 

In [ ]:
# Step 1: Handling Missing Values

# Check missing values
missing_values = df.isnull().sum()

# Drop 'Postal Code' since it has too many missing values and may not be crucial
df_cleaned = df.drop(columns=['Postal Code'])

# Step 2: Removing Duplicates
df_cleaned = df_cleaned.drop_duplicates()

# Step 3: Convert Date Columns to Datetime Format
df_cleaned['Order Date'] = pd.to_datetime(df_cleaned['Order Date'], errors='coerce', dayfirst=True)
df_cleaned['Ship Date'] = pd.to_datetime(df_cleaned['Ship Date'], errors='coerce', dayfirst=True)

# Check updated missing values after cleaning
cleaned_missing_values = df_cleaned.isnull().sum()


In [ ]:
# Display missing values before and after cleaning
missing_values, cleaned_missing_values

(Row ID                0
Order ID              0
Order Date            0
Ship Date             0
Ship Mode             0
Customer ID           0
Customer Name         0
Segment               0
City                  0
State                 0
Country               0
Postal Code       41296
Market                0
Region                0
Product ID            0
Category              0
Sub-Category          0
Product Name          0
Sales                 0
Quantity              0
Discount              0
Profit                0
Shipping Cost         0
Order Priority        0
dtype: int64, Row ID            0
Order ID          0
Order Date        0
Ship Date         0
Ship Mode         0
Customer ID       0
Customer Name     0
Segment           0
City              0
State             0
Country           0
Market            0
Region            0
Product ID        0
Category          0
Sub-Category      0
Product Name      0
Sales             0
Quantity          0
Discount          0
Profit   

In [ ]:
# Step 4: Detect and Handle Outliers using IQR method
numeric_columns = ['Sales', 'Profit', 'Discount', 'Shipping Cost', 'Quantity']

# Function to remove outliers using IQR
def remove_outliers_iqr(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

# Apply IQR method to remove outliers
df_no_outliers = remove_outliers_iqr(df_cleaned, numeric_columns)

# Display the number of rows before and after outlier removal
original_rows = df_cleaned.shape[0]
cleaned_rows = df_no_outliers.shape[0]


In [ ]:
original_rows, cleaned_rows

(51290, 30991)


In [ ]:
# Step 5: Statistical Analysis

# Compute summary statistics
summary_statistics = df_no_outliers[numeric_columns].describe()

# Compute correlations between numerical variables
correlation_matrix = df_no_outliers[numeric_columns].corr()

In [ ]:
# Step 6: Data Visualization - Histograms for numerical features
numeric_columns = ['Sales', 'Profit', 'Discount', 'Shipping Cost', 'Quantity']

# Plot histograms for each numeric column
for col in numeric_columns:
    plt.figure(figsize=(8, 5))
    plt.hist(df_no_outliers[col], bins=30, edgecolor='black', alpha=0.7)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()


In [ ]:
# Step 7: Boxplots for Outlier Detection

# Plot boxplots for each numeric column
for col in numeric_columns:
    plt.figure(figsize=(8, 5))
    plt.boxplot(df_no_outliers[col], vert=False, patch_artist=True)
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.show()


In [ ]:
# Step 8: Heatmap for Correlation Analysis
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap of Numerical Features")
plt.show()
